In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import os
import time

In [11]:
JAR_PATH = "/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/spark-sql-kafka-0-10_2.12-3.2.1.jar,/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/kafka-clients-3.1.0.jar"

kafka_topic_name = "songTopic"
kafka_bootstrap_servers = 'localhost:9092'

In [12]:
spark = SparkSession \
        .builder \
        .appName("Spotify Streaming Reccomendation System") \
        .master("local[*]") \
        .config("spark.jars", JAR_PATH) \
        .config("spark.executor.extraClassPath", JAR_PATH) \
        .config("spark.executor.extraLibrary", JAR_PATH) \
        .config("spark.driver.extraClassPath", JAR_PATH) \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [13]:
# Construct a streaming DataFrame that reads from test-topic
songs_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "latest") \
        .load()

songs_df1 = songs_df.selectExpr("CAST(value AS STRING)", "timestamp")


songs_schema_string = "order_id INT,id STRING, name STRING,popularity INT, duration_ms DOUBLE, explicit INT, " \
                           + "artists STRING, id_artists STRING, release_date STRING, " \
                           + "danceability DOUBLE," \
                           + "energy DOUBLE, key INT, loudness DOUBLE, " \
                           + "mode INT," \
                           + "speechiness DOUBLE," \
                           + "acousticness DOUBLE, instrumentalness DOUBLE, liveness DOUBLE, " \
                           + "valence DOUBLE, tempo DOUBLE, time_signature DOUBLE"



songs_df2 = songs_df1 \
        .select(from_csv(col("value"), songs_schema_string) \
                .alias("song"), "timestamp")


songs_df3 = songs_df2.select("song.*", "timestamp")


AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        